# First version to download Modern Slavery Statement PDFs.

Cannot handle HTML format slavery statements

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import pandas as pd
import numpy as np
from os import getcwd, path
import urllib
from requests.exceptions import HTTPError
import pickle
from time import sleep
import functools
import requests
from func_timeout import func_timeout, FunctionTimedOut

In [3]:
PROJECT_NAME = 'modern_slavery_registry'
PROJECT_PATH = f"{getcwd()[:getcwd().find(PROJECT_NAME)]}{PROJECT_NAME}"
DATA_PATH = f"{PROJECT_PATH}\\data"
SHEET_PATH = f"{DATA_PATH}\\sheets"
PICKLE_PATH = f"{DATA_PATH}\\pickles"
PDF_PATH = f"{DATA_PATH}\\pdfs"
MAX_DOWNLOAD_TIME = 5

In [4]:
def save_pickle(obj, file_name):
    pickle.dump(obj, file=open(f"{file_name}.pickle","wb"))
    
def load_pickle(file_name):
    return pickle.load(file=open(f"{file_name}.pickle","rb"))

In [5]:
pickle_name = f"{PICKLE_PATH}\\saved_SIDs"
if path.exists(f"{pickle_name}.pickle"):
    saved_SIDs = load_pickle(pickle_name)
    print(f"{pickle_name} exists, found - {len(saved_SIDs)}")
else:
    saved_SIDs = []
    save_pickle(saved_SIDs, file_name=pickle_name)
    
    
pickle_name = f"{PICKLE_PATH}\\failed_SIDs"
if path.exists(f"{pickle_name}.pickle"):
    failed_SIDs = load_pickle(pickle_name)
    print(f"{pickle_name} exists, found - {len(failed_SIDs)}")
else:
    failed_SIDs = []
    save_pickle(failed_SIDs, file_name=pickle_name)
    print(f"{pickle_name} created")
    
pickle_name = f"{PICKLE_PATH}\\pdf_not_found_SIDs"
if path.exists(f"{pickle_name}.pickle"):
    pdf_not_found_SIDs = load_pickle(pickle_name)
    print(f"{pickle_name} exists, found - {len(pdf_not_found_SIDs)}")
else:
    pdf_not_found_SIDs = []
    save_pickle(pdf_not_found_SIDs, file_name=pickle_name)
    print(f"{pickle_name} created")
    
pickle_name = f"{PICKLE_PATH}\\timeout_SIDs"
if path.exists(f"{pickle_name}.pickle"):
    timeout_SIDs = load_pickle(pickle_name)
    print(f"{pickle_name} exists, found - {len(timeout_SIDs)}")
else:
    timeout_SIDs = []
    save_pickle(timeout_SIDs, file_name=pickle_name)
    print(f"{pickle_name} created")

E:\Projects\modern_slavery_registry\data\pickles\saved_SIDs exists, found - 11044
E:\Projects\modern_slavery_registry\data\pickles\failed_SIDs exists, found - 4020
E:\Projects\modern_slavery_registry\data\pickles\pdf_not_found_SIDs exists, found - 2471
E:\Projects\modern_slavery_registry\data\pickles\timeout_SIDs exists, found - 264


In [6]:
print(f"Total SIDs worked on : {len(saved_SIDs + failed_SIDs + pdf_not_found_SIDs + timeout_SIDs)}")

Total SIDs worked on : 17799


In [7]:
df = pd.read_csv(f"{DATA_PATH}\modernslaveryregistry-2020-09-14.csv")
df.head()

,Company ID,Company,Is Publisher,Statement ID,URL,Override URL,Companies House Number,Industry,HQ,Is Also Covered,UK Modern Slavery Act,California Transparency in Supply Chains Act,Australia Modern Slavery Act,Period Covered
0,7676,"""K"" Line Holding Europe Limited",True,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,05005018,Marine,United Kingdom,False,True,False,False,2018-2019
1,28660,"""K"" Line Bulk Shipping (UK) Limited",False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,04830352,Marine,United Kingdom,True,True,False,False,2018-2019
2,28659,"""K"" Line (Europe) Limited",False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,05639474,Marine,United Kingdom,True,True,False,False,2018-2019
3,28661,"""K"" Line LNG Shipping Limited",False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,NaN,Marine,United Kingdom,True,True,False,False,2018-2019
4,28658,Polar LNG Shipping (UK) Limited,False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,02205323,Marine,United Kingdom,True,True,False,False,2018-2019


In [8]:
TOTAL_UNIQUE_SIDs = len(np.unique(df['Statement ID'].values))
print(f"Total enteries : {len(df)}, "
      f"total unique statements : {TOTAL_UNIQUE_SIDs}")

Total enteries : 27531, total unique statements : 17799


In [9]:
def download_pdf(url, sid):
    response = urllib.request.urlopen(url=url)
    file_name = f"SID-{statement_id}"
    file = open(f"{PDF_PATH}\\{file_name}.pdf", 'wb')
    file.write(response.read())
    file.close()

In [10]:
def get_pdf_url_from_sid(sid):
    """Returns PDF url for input sid."""
    return df[df['Statement ID']==sid]['URL'].values[0]

In [12]:
# # to get not saved pdfs
# new_df = df[~df['Statement ID'].isin(saved_SIDs)].copy()
new_df = df.copy()
new_df = new_df.drop_duplicates(
    subset=['Statement ID', 'URL']
)[['Statement ID', 'URL']].values
# failed_SIDs = []
# pdf_not_found_SIDs = []
# timeout_SIDs = []
# print(f"Try again for - {len(new_df)}")

In [ ]:
for i, row in enumerate(new_df):
    print(f"\n No. {i}/{TOTAL_UNIQUE_SIDs}")
    sid = row[0]
    url = row[1]
    if sid in saved_SIDs:
        print(f"Already downloaded for SID - {sid}, "
              f"{len(saved_SIDs)}/{TOTAL_UNIQUE_SIDs}")
    elif sid in pdf_not_found_SIDs:
        print(f"PDF not found for SID - {sid}, "
              f"{len(pdf_not_found_SIDs)}/{TOTAL_UNIQUE_SIDs}")
    elif sid in failed_SIDs:
        print(f"Already failed downloading for SID - {sid}, "
              f"{len(failed_SIDs)}/{TOTAL_UNIQUE_SIDs}")
    elif sid in timeout_SIDs:
        print(f"Already timeout downloading for SID - {sid}, " 
              f"{len(timeout_SIDs)}/{TOTAL_UNIQUE_SIDs}")
    else:
        try:
            doitReturnValue = func_timeout(timeout=MAX_DOWNLOAD_TIME,
                                           func=download_pdf, 
                                           args=[url, sid]) # if timeout
            saved_SIDs.append(int(sid))
            save_pickle(saved_SIDs, file_name=f"{PICKLE_PATH}\\saved_SIDs")
            print(f"Downloaded - {len(saved_SIDs)} / {TOTAL_UNIQUE_SIDs}")
            sleep(1)
        except FunctionTimedOut: # when more than MAX_DOWNLOAD_TIME
            timeout_SIDs.append(int(sid))
            save_pickle(timeout_SIDs, file_name=f"{PICKLE_PATH}\\timeout_SIDs")
            print(f"Downloading timeout - {len(timeout_SIDs)} / {TOTAL_UNIQUE_SIDs}")
        except urllib.error.HTTPError as HTTPErrror:
            if 'HTTP Error 404: Not Found' in str(HTTPErrror):
                pdf_not_found_SIDs.append(int(sid))
                save_pickle(pdf_not_found_SIDs, file_name=f"{PICKLE_PATH}\\pdf_not_found_SIDs")
                print(f"PDF not found - {len(pdf_not_found_SIDs)} / {TOTAL_UNIQUE_SIDs}")
            else:
                failed_SIDs.append(int(sid))
                save_pickle(failed_SIDs, file_name=f"{PICKLE_PATH}\\failed_SIDs")
                print(f"Downloading failed - {len(failed_SIDs)} / {TOTAL_UNIQUE_SIDs}")
        except:
            failed_SIDs.append(int(sid))
            save_pickle(failed_SIDs, file_name=f"{PICKLE_PATH}\\failed_SIDs")
            print(f"Downloading failed - {len(failed_SIDs)} / {TOTAL_UNIQUE_SIDs}")

In [52]:
random_sid = np.random.choice(pdf_not_found_SIDs)
print(f"{random_sid}-{get_pdfurl(random_sid)}")

10370-https://www.derwentlondon.com/uploads/downloads/investors/Corporate_Governance/170412_Derwent_London_Modern_Slavery_Statement_2017.pdf


In [47]:
print(f"Total PDFs available       : {TOTAL_UNIQUE_SIDs}")
print(f"Total PDFs downloaded      : {len(saved_SIDs)}")
print(f"Total PDFs not found       : {len(pdf_not_found_SIDs)}")
print(f"Total PDFs failed download : {len(failed_SIDs)}")
print(f"Total PDFs timeout download: {len(timeout_SIDs)}")
print(f"Total PDFs worked on       : {len(saved_SIDs) + len(pdf_not_found_SIDs) + len(failed_SIDs) + len(timeout_SIDs)}")

Total PDFs available       : 17799
Total PDFs downloaded      : 11044
Total PDFs not found       : 2471
Total PDFs failed download : 4020
Total PDFs timeout download: 264
Total PDFs worked on       : 17799


In [45]:
df[df['Statement ID'].isin(
    pdf_not_found_SIDs)
  ].to_excel(f"{SHEET_PATH}\modernslaveryregistry_pdf_not_found.xlsx", 
             index=False)

In [78]:
random_sid = np.random.choice(pdf_not_found_SIDs)
print(f"{random_sid}-{get_pdf_url_from_sid(random_sid)}")

11316-https://www.laterooms.com/en/static/modernslavery


In [81]:
sid = 30402
sid = 44998
# sid = 11316
# download_pdf(url=get_pdf_url_from_sid(sid=sid), sid=sid)

open('example.pdf', 
     'wb').write(requests.get(get_pdf_url_from_sid(sid=sid), 
                              allow_redirects=True).content)

110402

In [ ]:
for i, sid in enumerate(failed_SIDs):
    print(f"Running sid-{i+1} out of {len(failed_SIDs)}")
    url = get_pdf_url_from_sid(sid)
    file_name = f"SID-{sid}"
    try:
        open(f"{PDF_PATH}\\{file_name}.pdf", 
             'wb').write(requests.get(url, 
                                      timeout=5,
                                      allow_redirects=True).content)
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
    else:
        print('Success!')
        